# Skeleton Simulation

#### Imports

In [1]:
import sys
import math
import warnings

import psycopg2
import wrds
import gzip

import seaborn as sns
import os
import quandl
import json
import zipfile
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import functools
import requests
import io

import urllib.request
from urllib.error import HTTPError
from html_table_parser.parser import HTMLTableParser
from bs4 import BeautifulSoup
import re

import plotnine as p9
from plotnine import ggplot, scale_x_date, guides, guide_legend, geom_bar, scale_y_continuous, \
    scale_color_identity, geom_line, geom_point, labs, theme_minimal, theme, element_blank, element_text, \
        geom_ribbon, geom_hline, aes, scale_size_manual, scale_color_manual, ggtitle

from datetime import datetime
import datetime

import pandas as pd
import pandas_market_calendars as mcal
from pandas.plotting import autocorrelation_plot
import numpy as np
from numpy import cumsum, log, polyfit, sqrt, std, subtract
import scipy as sp
from scipy.stats import norm
import scipy.stats as stats

from statsmodels.tsa.stattools import coint
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from statsmodels.tsa.stattools import acf
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant

from collections import deque
from bisect import insort, bisect_left
from itertools import islice


### Retrieve data

In [2]:
csv_file_path = 'combinedata.csv'
data = pd.read_csv(csv_file_path)

In [6]:
display(data.head(200))

,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,expiry_indicator,close,adj_open,adj_close,adj_volume
0,2018-01-02,2018-01-03,2017-12-28,C,235.0,33.59,33.81,0.0,187.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0
1,2018-01-02,2018-01-03,2018-01-02,C,240.0,28.59,28.76,1.0,88.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0
2,2018-01-02,2018-01-03,2017-12-27,C,242.5,26.09,26.32,0.0,2.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0
3,2018-01-02,2018-01-03,2018-01-02,C,245.0,23.59,23.81,12.0,58.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0
4,2018-01-02,2018-01-03,NaN,C,247.5,21.08,21.32,0.0,0.0,NaN,NaN,NaN,NaN,NaN,w,268.77,242.053393,242.893856,86655749.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2018-01-02,2018-01-05,2018-01-02,P,268.5,0.53,0.55,4726.0,2801.0,0.069933,-0.429563,0.231414,9.563643,-39.13638,w,268.77,242.053393,242.893856,86655749.0
196,2018-01-02,2018-01-05,2018-01-02,P,269.0,0.73,0.76,4684.0,5112.0,0.065758,-0.549924,0.248915,9.635980,-36.64628,w,268.77,242.053393,242.893856,86655749.0
197,2018-01-02,2018-01-05,2018-01-02,P,269.5,1.02,1.07,357.0,1231.0,0.065012,-0.671967,0.231273,8.791567,-32.57904,w,268.77,242.053393,242.893856,86655749.0
198,2018-01-02,2018-01-05,2018-01-02,P,270.0,1.38,1.45,767.0,11745.0,0.065794,-0.775328,0.191777,7.278910,-26.83571,w,268.77,242.053393,242.893856,86655749.0


### Graphs

We will graph Gamma over time, combined gamma over time, delta over time for individual contracts and some zero-gamma pairs.

We will also graph the gamma for ATM options of different expiry over time.

### Simulation

This simulation code simulates trading with respect to pairs of individual contracts, with an initial delta of as close to 1 as possible.

The option contracts are opened at varying expiries, and are held until 1 week before expiry. 

Position in SPY is taken daily to rebalance the option delta as the underlying moves, and P&L is tracked from the option premium and SPY rebalances.
- Obtain the zero-delta pairs for the same expiry date at the interval (2 week, 1 month, 2 month), usually somewhat symmetrical about the ATM strike (how many do we want)
- Initial balance of stock for the remaining delta
- Every day, we will rebalance the position according to delta of the contracts we hold. 
- Save the daily position, and calculate daily PL vectorized.
- We will consider the close as our trading price for now.

Other considerations and parameters are described in ```outline.md```. This simulator does not actually outline the requirements or thresholds for opening a position, rather merely simulates all of the possible individual positions for our time frame.

In [ ]:
class Simulate:
    def __init__(self, data):
        """
        Initialize the simulation with the provided data.
        
        :param data: A DataFrame containing options data.
        """
        self.data = data
        self.positions = None
        self.daily_pl = None

    def find_zero_delta_pairs(self, intervals):
        """
        Identify zero-delta pairs for specified intervals.

        :param intervals: A list of intervals to find zero-delta pairs for.
        """

    def calculate_initial_position(self):
        """
        Calculate the initial stock position needed to balance the delta.
        """

    def rebalance_daily(self):
        """
        Rebalance the stock position daily based on the option contracts' delta.
        """

    def calculate_daily_pl(self):
        """
        Calculate daily profit and loss.
        """

    def run_simulation(self):
        """
        Run the full simulation, executing all necessary steps.
        """

    def save_results(self, filename):
        """
        Save the daily positions and P&L calculations to a file.

        :param filename: The name of the file to save the results to.
        """

simulate = Simulate(data)
simulate.run_simulation()
